In [ ]:
#  This is filename:  Calc_Moments.ipynb

#  The purpose of this notebook is to compute
#  commonly used moments for the cyclical 
#  components of time series, which result
#  from simulation results.  But, bviously, it
#  can be applied to any data set.

#  Inputs are the matrices Mat_cycle and
#  Mat_cycle_named.  Or other matrices
#  containing detrended time series.

#  Outputs are the matrices Mom_Table_xx  
#  and  Table_xx  which can be downloaded into 
#  excel or used to make plots.


import numpy as np
import numpy.matlib as npm
from numpy import hstack, vstack, zeros, dot, eye, kron

import scipy as sp
from scipy import linalg as la
from scipy.linalg import null_space
import scipy.optimize as opt

import matplotlib.pyplot as plt
import pylab

import pandas as pd

import statsmodels.api as sm

from IPython.display import display, HTML 

np.set_printoptions(precision=2, suppress=False)
pd.set_option('display.float_format', '{:.2e}'.format)


print('  ')
print('  ')
print('  ')
print('Here are the moments for the simulated series.')
print('You can also dowload these in tables Mom_Table_01 to _04.')
print('  ')
print('  ')


# Some preliminaries

##  USER INPUT:  Matrices containing detrended
##  time series

Input_momat =  Mat_cycle
Input_momat_named = Mat_cycle_named

Varnames = np.array(VARNAMES)

##  USER INPUT:  Row location of Output or GDP in Mat_cycle

output_index =  3  # This is different for each model

n_obs = Input_momat.shape[1]

n_leads = 5  #  number of leads or autocorrelations
n_lags = 5  #  number of lags or autocorrelations


#  Table 01:  Mean, Standard Deviation, Coefficient of Variation

#  USER NOTE:  "1"  calculates the mean by rows, "0" by columns

cycle_MEAN = Input_momat.mean(1) 
cycle_STDEV = Input_momat.std(1)  
cycle_CEOFVAR = cycle_STDEV/cycle_MEAN

col_01_labels = Varnames
row_01_labels = ['Mean', 'Std Deviation', 'Coeff of Variation']

print(' ')
print('Table 01:  Mean, Standard Deviation, Coefficient of Variation')
print(' ')

Mom_Table_01 = np.hstack((cycle_MEAN, cycle_STDEV, cycle_CEOFVAR))
Mom_Table_01 = pd.DataFrame(Mom_Table_01, index = col_01_labels, columns = row_01_labels)

display(HTML(Mom_Table_01.to_html()))
print(' ')

print(' ')



#  Table 02:  Correlations with Output, Std Deviation Relative to Output

cycle_GDP_correl = np.zeros(Input_momat.shape[0]).reshape(Input_momat.shape[0],1)

variable_count = 0
temp_a = 0
temp_b = 0
temp_c = 0

for variable_count in range(0,Input_momat.shape[0]):
    temp_a = np.ndarray.flatten(np.array(Input_momat[variable_count,:]))
    temp_b =np.ndarray.flatten(np.array(Input_momat[output_index,:]))
    temp_c  = np.corrcoef(temp_a, temp_b)
    cycle_GDP_correl[variable_count,0] = temp_c[0,1]
    
cycle_GDP_relstd = np.zeros(Input_momat.shape[0]).reshape(Input_momat.shape[0],1)    
cycle_GDP_relstd = cycle_STDEV/cycle_STDEV[output_index,0]

col_02_labels = Varnames
row_02_labels = ['Correlation with GDP', 'Std Dev Relative to GDP']

print(' ')
print('Table 02:  Cross Correlation with Output, Standard Deviation Relative to Output')
print(' ')

Mom_Table_02 = np.hstack((cycle_GDP_correl, cycle_GDP_relstd))
Mom_Table_02 = pd.DataFrame(Mom_Table_02, index = col_02_labels, columns = row_02_labels)

display(HTML(Mom_Table_02.to_html()))
print(' ')

print(' ')



#  Table 03:  Autocorrelations  (Not Cross Autocorrelations)

t_labels = np.arange(n_leads, -(n_lags+1), -1)
var_labels = Varnames

coeff_collector_all = (np.zeros((n_leads+n_lags+1)*Input_momat.shape[0])).reshape((n_leads+n_lags+1),Input_momat.shape[0])

#  For leading autocorrelations

var_count = 0

for var_count in range(0,Input_momat.shape[0]):
    
    leads_count = 0
    
    for leads_count in range(1,n_leads+1):
        
        var_now = Input_momat[var_count,leads_count:n_obs].reshape((n_obs-leads_count),1)
        var_now = np.ndarray.flatten(np.array(var_now))
        
        var_lead = Input_momat[var_count,0:(n_obs-leads_count)].reshape((n_obs-leads_count),1)
        var_lead = np.ndarray.flatten(np.array(var_lead))
        
        coeff_collector_all[leads_count-1, var_count] = np.corrcoef(var_lead, var_now)[0,1]
        
#  For contemporaneous correlations       
        
for var_count in range(0,Input_momat.shape[0]):
    coeff_collector_all[n_leads, var_count] = np.corrcoef(Input_momat[var_count,:], Input_momat[var_count,:])[0,1]
        
#  For lagging autocorrelations

var_count = 0

for var_count in range(0,Input_momat.shape[0]):
    
    lags_count = 0
    
    for lags_count in range(1,n_lags+1): 
        
        var_now = Input_momat[var_count,0:(n_obs-lags_count)].reshape((n_obs-lags_count),1)
        var_now = np.ndarray.flatten(np.array(var_now))
        
        var_lag = Input_momat[var_count,lags_count:n_obs].reshape((n_obs-lags_count),1)
        var_lag = np.ndarray.flatten(np.array(var_lag))
        
        coeff_collector_all[n_leads+lags_count, var_count] = np.corrcoef(var_lag, var_now)[0,1]

print('  ')       
print('Table 03:  Autocorrelations  t + Leads  t=0  t - Lags  (Not Cross Autocorrelations)')
print(' ')

Mom_Table_03 = coeff_collector_all
Mom_Table_03= pd.DataFrame(Mom_Table_03, index = t_labels, columns = var_labels)

display(HTML(Mom_Table_03.to_html()))
print(' ')

print(' ')                                                   
                                                        

#  Table 04:  Cross Autocorrelation with Output

t_labels = np.arange(n_leads, -(n_lags+1), -1)
var_labels = Varnames

coeff_collector_all = (np.zeros((n_leads+n_lags+1)*Input_momat.shape[0])).reshape((n_leads+n_lags+1),Input_momat.shape[0])

#  For leading autocorrelations

var_count = 0

for var_count in range(0,Input_momat.shape[0]):
        
    leads_count = 0
    
    for leads_count in range(1,n_leads+1):
        
        var_now = Input_momat[output_index,leads_count:n_obs].reshape((n_obs-leads_count),1)
        var_now = np.ndarray.flatten(np.array(var_now))
        
        var_lead = Input_momat[var_count,0:(n_obs-leads_count)].reshape((n_obs-leads_count),1)
        var_lead = np.ndarray.flatten(np.array(var_lead))
        
        coeff_collector_all[leads_count-1, var_count] = np.corrcoef(var_lead, var_now)[0,1]
        
#  For contemporaneous correlations       
        
for var_count in range(0,Input_momat.shape[0]):
    coeff_collector_all[n_leads, var_count] = np.corrcoef(Input_momat[var_count,:], Input_momat[output_index,:])[0,1]
    
#  For lagging autocorrelations

var_count = 0

for var_count in range(0,Input_momat.shape[0]):
    
    lags_count = 0
    
    for lags_count in range(1,n_lags+1): 
        
        var_now = Input_momat[output_index,0:(n_obs-lags_count)].reshape((n_obs-lags_count),1)
        var_now = np.ndarray.flatten(np.array(var_now))
        
        var_lag = Input_momat[var_count,lags_count:n_obs].reshape((n_obs-lags_count),1)
        var_lag = np.ndarray.flatten(np.array(var_lag))
        
        coeff_collector_all[n_leads+lags_count, var_count] = np.corrcoef(var_lag, var_now)[0,1]
        
print('  ')    
print('Table 04:  Cross Autocorrelations  t + Leads  t=0  t - Lags')
print(' ')

Mom_Table_04 = coeff_collector_all
Mom_Table_04= pd.DataFrame(Mom_Table_04, index = t_labels, columns = var_labels)

display(HTML(Mom_Table_04.to_html()))
print(' ')

print(' ')
               


